In [1]:
import pandas as pd
import numpy as np

In [2]:
def contains_total(data_list):
    # Iterate over each element in the list
    for item in data_list:
        # Check if "TOTAL" is in the current item (case-insensitive)
        if "TOTAL" in item.upper():
            return True
    return False

def is_multiple_of_15(time_str):
    hour, minute = map(int, time_str.split(':'))
    return minute % 15 == 0
    
def read_excel_until_total(file_path, start_row=0):
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # Initialize a list to store data from all sheets
    all_data = []

    # Iterate through each sheet in the Excel file
    for i, sheet_name in enumerate(xls.sheet_names):
        # Read the sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name,header=11)

        # # Filter rows starting from `start_row`
        filtered_data = []

        for index, row in df.iterrows():
            if contains_total(row.astype(str).values):
                break
            filtered_data.append(row)

        # Convert the list of rows back into a DataFrame
        if filtered_data:  # Proceed only if there is data to add
            sheet_data = pd.DataFrame(filtered_data)
            sheet_data['SHEET NAME'] = sheet_name  # Add the sheet name to the DataFrame

            # Append the filtered data from this sheet to all_data
            all_data.append(sheet_data)

    # Concatenate all the data from all sheets
    result_df = pd.concat(all_data, ignore_index=True)
    return result_df

def penyesuaian_format(df):
    # Memastikan bahwa SHEET NAME diformat dengan dua digit (misalnya '01', '02', ..., '31')
    df['SHEET NAME'] = df['SHEET NAME'].str.zfill(2)
    
    # Menambahkan tanggal, bulan, dan tahun ke waktu
    df['WAKTU'] = thn_bln + df['SHEET NAME'] + ' ' + df['WAKTU']  # Tambahkan tanggal default
    
    # Mengubah kolom 'WAKTU' menjadi datetime dengan format lengkap
    df['WAKTU'] = pd.to_datetime(df['WAKTU'], format='%Y-%m-%d %H:%M')
    
    # Filter data berdasarkan apakah menit adalah kelipatan dari 15
    df = df[df['WAKTU'].dt.minute % 15 == 0]
    return df

def mencari_data_hilang(df, nama_file):    
    # Menghasilkan rentang tanggal untuk bulan yang ditentukan
    month_start = pd.Timestamp(f'{thn_bln}01')
    month_end = month_start + pd.offsets.MonthEnd(0)
    
    # Membuat rentang tanggal untuk seluruh bulan
    dates_range = pd.date_range(start=month_start, end=month_end, freq='D')
    
    # Inisialisasi list untuk menyimpan DataFrame dari waktu yang hilang
    missing_times_list = []

    # menghapus jumlah == 0
    df = df[df['JUMLAH'] != 0]
    
    # Loop melalui setiap hari dalam rentang bulan
    for date in dates_range:
        day = date.date()
        
        # Filter data untuk hari ini
        daily_data = df[df['WAKTU'].dt.date == day]
        
        # Buat rentang waktu yang lengkap untuk hari ini
        full_daily_range = pd.date_range(start=f'{day} 00:00', end=f'{day} 23:45', freq='15T')
        
        # Identifikasi waktu yang hilang
        missing_daily_times = full_daily_range[~full_daily_range.isin(daily_data['WAKTU'])]
        
        # Jika ada waktu yang hilang, tambahkan ke list missing_times_list
        if not missing_daily_times.empty:
            missing_times_df = pd.DataFrame({
                'WAKTU': missing_daily_times,
                'MOTOR': np.nan,
                'MOBIL': np.nan,
                'TRUK/BUS': np.nan,
                'JUMLAH': np.nan,
                'GAP (s)': np.nan,
                'HEADWAY (s)': np.nan,
                'AVG. SPEED (Km/h)': np.nan,
                'OCCUPANCY (%)': np.nan,
                'SHEET NAME': str(day.day).zfill(2)
            })
            missing_times_list.append(missing_times_df)
    
    # Menggabungkan semua DataFrame dari missing_times_list menjadi satu DataFrame
    missing_times = pd.concat(missing_times_list, ignore_index=True) if missing_times_list else pd.DataFrame()
    
    # Menampilkan data yang hilang
    if missing_times.empty:
        print("Tidak ada data yang hilang.")
    else:
        print("Ada data yang hilang")
    #     print(missing_times)
    
    return missing_times, df

def pisahkan_kolom_waktu(df):
    # Pastikan kolom WAKTU sudah dalam format datetime
    df['WAKTU'] = pd.to_datetime(df['WAKTU'])

    # Pisahkan kolom WAKTU menjadi beberapa kolom
    df['TAHUN'] = df['WAKTU'].dt.year
    df['BULAN'] = df['WAKTU'].dt.month
    df['TANGGAL'] = df['WAKTU'].dt.day
    df['HARI'] = df['WAKTU'].dt.day_name()  # Ini akan memberikan nama hari dalam format string
    df['JAM'] = df['WAKTU'].dt.strftime('%H:%M')

    # Daftar kolom yang diinginkan dalam urutan baru
    new_column_order = [
        'WAKTU', 'TAHUN', 'BULAN', 'PEKAN', 'TANGGAL', 'HARI', 'JAM', 
        'MOTOR', 'MOBIL', 'TRUK/BUS', 'JUMLAH', 
        'SHEET NAME', 'GAP (s)', 'HEADWAY (s)', 
        'AVG. SPEED (Km/h)', 'OCCUPANCY (%)'
    ]
    
    # Menyusun ulang kolom berdasarkan urutan yang diinginkan
    df = df.reindex(columns=new_column_order)
    return df
    
def hapus_kolom(df):
    # Misalkan df adalah DataFrame yang sudah ada
    df = df.drop(columns=['SHEET NAME', 'GAP (s)', 'HEADWAY (s)', 'AVG. SPEED (Km/h)', 'OCCUPANCY (%)'])
    return df

def perhitungan_smp(df):
    df['JUMLAH'] = df['MOTOR'] + df['MOBIL'] + df['TRUK/BUS']
    df['SMP_MTR'] = df['MOTOR'] * 0.8
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])
    
    return df

# Fungsi untuk menghitung minggu ke berapa dalam bulan
def week_of_month(dt):
    """Menghitung minggu ke berapa dalam bulan dari tanggal tertentu."""
    day_of_month = dt.day
    if day_of_month <= 7:
        return 1
    elif day_of_month <= 14:
        return 2
    elif day_of_month <= 21:
        return 3
    elif day_of_month <= 28:
        return 4
    else:
        return 5

In [3]:
# data files
nama_files = ['01 Januari', '02 Februari', '03 Maret', '04 April', '05 Mei',
              '06 Juni', '07 Juli', '08 Agustus','09 September',
              '10 Oktober', '11 November', '12 Desember']

# Inisialisasi list untuk menampung DataFrame
all_dfs = []
all_miss = []

for nama_file in nama_files:
    bulan = nama_file.split()[0]
    tahun = '2023'
    thn_bln = f'{tahun}-{bulan}-'
    
    nama_file = nama_file + '.xlsx'
    file_path = '../Data/DataLHR/Data Losarang/2023/' + nama_file 
    
    # Read the Excel file from the specified row until 'Total' is encountered
    result = read_excel_until_total(file_path, start_row=11)
    result = penyesuaian_format(result)
    missing_times, result = mencari_data_hilang(result, nama_file)
    missing_times = pisahkan_kolom_waktu(missing_times)
    missing_times = hapus_kolom(missing_times)
    missing_times = perhitungan_smp(missing_times)
    result = pisahkan_kolom_waktu(result)
    result = hapus_kolom(result)
    result = perhitungan_smp(result)

    missing_times['PEKAN'] = missing_times['WAKTU'].apply(week_of_month)
    result['PEKAN'] = result['WAKTU'].apply(week_of_month)
    
    df = result.copy()

    # Tambahkan DataFrame ke list
    all_dfs.append(result)
    all_miss.append(missing_times)
    
    # save the result to a new Excel file
    df.to_excel('../Data/Preprocessing/Data Losarang/2023/1_input_'+nama_file, index=False)
    missing_times.to_excel('../Data/Preprocessing/Data Losarang/2023/1_missing_times_'+nama_file, index=False)

# gabungkan data
df_combined = pd.concat(all_dfs, ignore_index=True)
df_combined_miss = pd.concat(all_miss, ignore_index=True)

Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang
Ada data yang hilang


In [4]:
df_combined.head(10)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,1,Sunday,00:00,85.0,24.0,4.0,113.0,68.0,24.0,10.0,102.0
1,2023-01-01 00:15:00,2023,1,1,1,Sunday,00:15,92.0,19.0,6.0,117.0,73.6,19.0,15.0,108.0
2,2023-01-01 00:30:00,2023,1,1,1,Sunday,00:30,78.0,16.0,6.0,100.0,62.4,16.0,15.0,93.0
3,2023-01-01 00:45:00,2023,1,1,1,Sunday,00:45,78.0,23.0,4.0,105.0,62.4,23.0,10.0,95.0
4,2023-01-01 01:00:00,2023,1,1,1,Sunday,01:00,76.0,14.0,6.0,96.0,60.8,14.0,15.0,90.0
5,2023-01-01 01:15:00,2023,1,1,1,Sunday,01:15,80.0,23.0,7.0,110.0,64.0,23.0,17.5,104.0
6,2023-01-01 01:30:00,2023,1,1,1,Sunday,01:30,55.0,10.0,5.0,70.0,44.0,10.0,12.5,66.0
7,2023-01-01 01:45:00,2023,1,1,1,Sunday,01:45,71.0,22.0,1.0,94.0,56.8,22.0,2.5,81.0
8,2023-01-01 02:00:00,2023,1,1,1,Sunday,02:00,59.0,22.0,3.0,84.0,47.2,22.0,7.5,77.0
9,2023-01-01 02:15:00,2023,1,1,1,Sunday,02:15,56.0,12.0,4.0,72.0,44.8,12.0,10.0,67.0


In [5]:
df_combined_miss.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-03 04:15:00,2023,1,1,3,Tuesday,04:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-03 05:15:00,2023,1,1,3,Tuesday,05:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-04 07:45:00,2023,1,1,4,Wednesday,07:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-05 00:00:00,2023,1,1,5,Thursday,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-05 07:30:00,2023,1,1,5,Thursday,07:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def imputing_data_row(df,missing):
    df_a = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=-15)]
    df_b = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=15)]
    MOTOR = -1
    MOBIL = -1
    TRUK = -1
    if(len(df_a) + len(df_b) == 2):
        result = pd.concat([df_a,df_b])
        MOTOR = round(result['MOTOR'].mean())
        MOBIL = round(result['MOBIL'].mean())
        TRUK = round(result['TRUK/BUS'].mean())
    return MOTOR, MOBIL, TRUK 

In [7]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_combined_miss.iterrows():
    MOTOR, MOBIL, TRUK = imputing_data_row(df_combined, missing)
    if (MOTOR != -1) & (np.isnan(missing['MOTOR'])):
        df_combined_miss.at[i, 'MOTOR'] = MOTOR
        df_combined_miss.at[i, 'MOBIL'] = MOBIL
        df_combined_miss.at[i, 'TRUK/BUS'] = TRUK

In [8]:
# Append these non-missing rows to df_combined
non_missing_motor = df_combined_miss[df_combined_miss['MOTOR'].notna()]
non_missing_motor_smp = non_missing_motor.copy()
non_missing_motor_smp = perhitungan_smp(non_missing_motor_smp)
df_combined = pd.concat([df_combined, non_missing_motor_smp], ignore_index=False)
df_combined =df_combined.sort_values('WAKTU')

# Update df_combined_miss to contain only missing 'MOTOR' entries
df_combined_miss = df_combined_miss[df_combined_miss['MOTOR'].isna()]

In [9]:
# Simpan hasil gabungan ke file Excel
df_combined.to_excel('../Data/Preprocessing/Data Losarang/2023/1_input_losarang_2023.xlsx', index=False)
df_combined_miss.to_excel('../Data/Preprocessing/Data Losarang/2023/1_missing_times_losarang_2023.xlsx', index=False)

In [10]:
len(df_combined),len(df_combined_miss), len(df_combined) + len(df_combined_miss)

(20909, 14131, 35040)

In [11]:
unique_values = df_combined['PEKAN'].unique()
print("Nilai Unik di PEKAN:", unique_values)

Nilai Unik di PEKAN: [1 2 3 4 5]
